In [1]:
import numpy as np
import pandas as pd
#from scipy import stats as st
import statistics as st
np.set_printoptions(precision=2)

Get files form xlsx file and delete nan row

In [3]:
df = pd.read_excel('Sample_Data.xlsx', sheet_name='Sheet1')
temp_numpy_array = df.values
temp=np.asanyarray(temp_numpy_array[:,1:])
temp_main_array=np.delete(temp,41,axis=0) ###delete nan row not NAN row


Funtion for deleting data

In [4]:
def remove_NAN(array):
    temp=[]
    for row in range(array.shape[0]-1):
        for column in range(array.shape[1]):
            if array[row,column]=="NAN":
                temp.append(row) 
                break  
    array=np.delete(array,temp,axis=0) 
    return array         

 Fill in the missing data or wrong data with the mode

In [5]:
def fill_with_mod(main_array):
    for column in range(0,6):
        max=main_array[main_array.shape[0]-1,column]
        mode=st.mode(main_array[:,column])
        for row in main_array:
            if row[column]>max or row[column]<0 :
                row[column]=mode 

    return main_array     

Identify outliers using IQR 

In [6]:
def iqr(array):
    for column in range(0,6):
        sorted=np.sort(array[:array.shape[0]-1,column])
        median=np.median(sorted)
        Q1=np.percentile(sorted,25) 
        Q3=np.percentile(sorted,75)
        IQR=Q3-Q1
        for row in range(array.shape[0]-1):
            if array[row,column]<(Q1-2.5*IQR) or array[row,column]>(Q3+2.5*IQR):
                ##print(row)
                array[row,column]=median ###chon ma on radif maxo dar aval dar nazar nagreftim
                
    return array

Normalizing data

In [ ]:
def normalized(array):
    for column in range(0,5):
        mean_value = np.mean(array[:array.shape[0]-1,column])
        std_value = np.std(array[:array.shape[0]-1,column]) ##peyda krdn enharaf meiar                    
        array[:array.shape[0]-1,column] = (array[:array.shape[0]-1,column] - mean_value) / std_value
        

    for i in range(array.shape[0]):
        for j in range(0,6):
            array[i,j]=np.round(array[i,j],decimals=2)

    return array   

Function for calculating correlation matrix

In [ ]:
def calculate_corr(array):
    temp_array=array[:array.shape[0],:]
    array_float = temp_array.astype(float)
    corr_matrix = np.corrcoef(array_float, rowvar=False)
    return corr_matrix

Implementation of the conditions mentioned in the project definition using the calculated matrix

In [ ]:
def final(array,corr_matrix):
    column_should_delete=[]
    for i in range(20):    
        max=0
        for row in range(corr_matrix.shape[0]-1):
            for column in range(corr_matrix.shape[1]-1):
                if row==column:
                    column=column+1
                    row=0
                    break
                else:
                    if corr_matrix[row,column]>max:
                        max=corr_matrix[row,column]
                        m_column=column
                        m_row=row       
        if corr_matrix[m_column,5]>corr_matrix[m_row,5]:
             #array=np.delete(array, m_row, axis=1)
             column_should_delete.append(m_row)
             corr_matrix[:,m_row]=0
             corr_matrix[m_row,:]=0
        else:
             #array=np.delete(array, m_column, axis=1)
             column_should_delete.append(m_column)
             corr_matrix[:,m_column]=0
             corr_matrix[m_column,:]=0


        if len(column_should_delete)==array.shape[1]-4:
            break    

    for i in column_should_delete:
        array=np.delete(array,i,axis=1)
    return array